# About

Run `tcpdump` to see if we can read the raw bytes off the wire and verify endianess of the TCP header.

In [8]:
! sudo tcpdump -nn -c1 -X tcp dst port 80

tcpdump: verbose output suppressed, use -v or -vv for full protocol decode
listening on eno1, link-type EN10MB (Ethernet), capture size 262144 bytes
14:17:35.632129 IP 10.10.138.125.49166 > 109.238.50.21.80: Flags [S], seq 3608769402, win 29200, options [mss 1460,sackOK,TS val 4978207 ecr 0,nop,wscale 7], length 0
	0x0000:  4500 003c 8565 4000 4006 80cc 0a0a 8a7d  E..<.e@.@......}
	0x0010:  6dee 3215 c00e 0050 d719 737a 0000 0000  m.2....P..sz....
	0x0020:  a002 7210 34b9 0000 0204 05b4 0402 080a  ..r.4...........
	0x0030:  004b f61f 0000 0000 0103 0307            .K..........
1 packet captured
2 packets received by filter
0 packets dropped by kernel
29 packets dropped by interface


In the above capture we pass the following options to the `tcpdump` command:

* `-nn` Do not resolve hostnames, don't convert protocol and port numbers etc. to names either.
* `-c` Capture _x_ packets and then stop (in our case we just capture 1).
* `tcp` Capture only TCP packets
* `dst port` Filter only packets going to port 80
* `-X` Print data of the packet in hex.

Note that the output of `tcpdump` will change everytime we run it. So for reference here is the ouput that we will use for the remainder of the text:

```
tcpdump: verbose output suppressed, use -v or -vv for full protocol decode
listening on eno1, link-type EN10MB (Ethernet), capture size 262144 bytes
14:17:35.632129 IP 10.10.138.125.49166 > 109.238.50.21.80: Flags [S], seq 3608769402, win 29200, options [mss 1460,sackOK,TS val 4978207 ecr 0,nop,wscale 7], length 0
	0x0000:  4500 003c 8565 4000 4006 80cc 0a0a 8a7d  E..<.e@.@......}
	0x0010:  6dee 3215 c00e 0050 d719 737a 0000 0000  m.2....P..sz....
	0x0020:  a002 7210 34b9 0000 0204 05b4 0402 080a  ..r.4...........
	0x0030:  004b f61f 0000 0000 0103 0307            .K..........
1 packet captured
2 packets received by filter
0 packets dropped by kernel
29 packets dropped by interface
```


The quick glance we see that the packet captured was going from:

* 10.10.138.125.49166, where 10.10.138.125 is the source IP and 49166 is the source port.

To: 
* 109.238.50.21.80, where 109.238.50.21 is the destination IP and 80 is the destination port.

Good with that in place we can look at the bytes of the packet, it was a bit hard to find but it seems that `tcpdump` prints also the IP header. See format [here](https://en.wikipedia.org/wiki/IPv4). 

The first four bits are the IP version used, which is 0x4, so we are using IP version 4. The length of the IP header in 32 bit words is the next four bits. Which is 0x5, so we konw that our TCP header will start $5\cdot32$ bits into the data. This is equivalent to skipping over 40 hex digits. 

This means that the first 32 bit word of the TCP header must be `c00e 0050`. 

Important note is that memory wise this is stored as (where memory is some pointer to our data):

* `memory[0] = c0;`
* `memory[1] = 0e;`
* `memory[2] = 00;`
* `memory[3] = 50;`

When we interpret the memory we have to be a bit careful, both the source an destination port stored in this 32 bit memory are stored using _network byte order_ or _big endian_ which means that the most significant byte comes first. 

* The first 16 bits are the source port `0xc00e` which equals 49166 in decimal.
* The second 16 bits are the destination port `0x0050` which equals 80 in decimal.

This matches our inital expectation.

Lets try to parse this with bitter:

In [1]:
! git clone git@github.com:steinwurf/bitter.git

Cloning into 'bitter'...
remote: Counting objects: 646, done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 646 (delta 42), reused 0 (delta 0), pack-reused 566
Receiving objects: 100% (646/646), 190.12 KiB | 0 bytes/s, done.
Resolving deltas: 100% (347/347), done.
Checking connectivity... done.


We also need to get be able to read the big endian numbers and convert them to the host manchines endianess. This can be done with our endian library:

In [9]:
! git clone git@github.com:steinwurf/endian.git

Cloning into 'endian'...
remote: Counting objects: 347, done.
remote: Total 347 (delta 0), reused 0 (delta 0), pack-reused 347
Receiving objects: 100% (347/347), 152.41 KiB | 0 bytes/s, done.
Resolving deltas: 100% (202/202), done.
Checking connectivity... done.


In [16]:
%%file parse_tcp_header.cpp

#include <iostream>

#include <bitter/reader.hpp>
#include <endian/big_endian.hpp>

int main()
{
    uint8_t data[] = {0xc0,0x0e,0x00,0x50};
    
    std::cout << "src port = " << endian::big_endian::get<uint16_t>(data) << std::endl;
    std::cout << "dest port = " << endian::big_endian::get<uint16_t>(data+2) << std::endl;
    
    return 0;
}

Overwriting parse_tcp_header.cpp


In [17]:
! g++ -std=c++11 -Ibitter/src -Iendian/src parse_tcp_header.cpp 

Lets run it:

In [18]:
! ./a.out

src port = 49166
dest port = 80
